# Notebook Terstruktur untuk STT dan Terjemahan

Notebook ini berisi kode untuk pipeline Speech-to-Text (STT) dan Terjemahan (Translation), serta evaluasi performanya menggunakan metrik WER dan BLEU.

## 1. Instalasi dan Import Library

In [ ]:
! pip install jiwer sacrebleu librosa tensorflow pandas numpy

import os
import random
import numpy as np
import librosa
import tensorflow as tf
import pickle
import pandas as pd
import sacrebleu
from jiwer import wer
from tensorflow.keras.models import load_model

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 45.1 MB/s eta 0:00:00


## 2. Memuat Model dan Tokenizer

Memuat model STT dan Terjemahan, serta tokenizer yang diperlukan.

In [ ]:
# Path ke model dan tokenizer Anda di Google Drive
stt_model_path = '/content/drive/MyDrive/databaru/stt.h5'
terjemahan_model_path = '/content/drive/MyDrive/databaru/terjemahmodel.h5'
stt_tokenizer_path = '/content/drive/MyDrive/databaru/tokenizer.pkl'
translation_source_tokenizer_path = '/content/drive/MyDrive/databaru/translation_source_tokenizer.pkl'
translation_target_tokenizer_path = '/content/drive/MyDrive/databaru/translation_target_tokenizer.pkl'

# Muat Model STT dan Terjemahan
try:
    stt_model = load_model(stt_model_path)
    terjemahan_model = load_model(terjemahan_model_path)
    print("Model STT dan Terjemahan berhasil dimuat.")
except Exception as e:
    print(f"Error memuat model: {e}")

# Muat Tokenizer untuk STT dan Terjemahan
try:
    with open(stt_tokenizer_path, 'rb') as f:
        stt_tokenizer = pickle.load(f)

    with open(translation_source_tokenizer_path, 'rb') as f:
        translation_source_tokenizer = pickle.load(f)

    with open(translation_target_tokenizer_path, 'rb') as f:
        translation_target_tokenizer = pickle.load(f)
    print("Tokenizer berhasil dimuat.")
except Exception as e:
    print(f"Error memuat tokenizer: {e}")

# Define sequence lengths for translation model (these values might need adjustment based on your training data)
src_length = 7  # Example source sequence length
tar_length = 10  # Example target sequence length
print(f"Panjang sekuens sumber (src_length): {src_length}")
print(f"Panjang sekuens target (tar_length): {tar_length}")

Model STT dan Terjemahan berhasil dimuat.
Tokenizer berhasil dimuat.
Panjang sekuens sumber (src_length): 7
Panjang sekuens target (tar_length): 10


## 3. Definisi Fungsi

Mendefinisikan fungsi-fungsi untuk ekstraksi fitur audio, prediksi STT, prediksi terjemahan, dan pipeline keseluruhan.

In [ ]:
# Fungsi untuk mengekstrak fitur audio (gabungan MFCC dan Mel-spectrogram)
def extract_audio_features(file_path, max_length=7, n_mels=128, n_mfcc=13):
    audio, sr = librosa.load(file_path, sr=None)  # Load dengan sample rate asli

    # Ekstrak Mel-spectrogram
    mel_spec = librosa.feature.melspectrogram(y=audio, sr=sr, n_fft=2048, n_mels=n_mels, fmax=sr/2)
    mel_spec = librosa.power_to_db(mel_spec, ref=np.max)
    mel_spec = mel_spec.T  # Transpose agar memiliki dimensi (time_steps, n_mels)

    # Ekstrak MFCC
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc)
    mfccs = mfccs.T # Transpose agar memiliki dimensi (time_steps, n_mfcc)


    # Gabungkan fitur
    combined_features = np.concatenate((mel_spec, mfccs), axis=1)

    # Padding atau pemotongan jika panjang timesteps tidak sesuai
    if combined_features.shape[0] < max_length:
        pad_width = max_length - combined_features.shape[0]
        combined_features = np.pad(combined_features, ((0, pad_width), (0, 0)), mode='constant')
    elif combined_features.shape[0] > max_length:
        combined_features = combined_features[:max_length, :]

    return combined_features

# Fungsi untuk mengonversi audio ke teks (STT)
def predict_audio_to_text(stt_model, audio_file):
    feature = extract_audio_features(audio_file)
    feature = np.expand_dims(feature, axis=0)  # Menambahkan dimensi batch
    pred = stt_model.predict(feature)
    pred = np.argmax(pred, axis=-1)

    # Decode hasil prediksi menjadi teks
    pred_text_list = []
    for seq in pred:
        decoded_seq = [stt_tokenizer.index_word.get(idx, '') for idx in seq if idx != 0]
        pred_text_list.append(' '.join(decoded_seq))

    return pred_text_list[0]

# Fungsi untuk tokenisasi dan encoding input teks
def encode_sequences(tokenizer, length, lines):
    X = tokenizer.texts_to_sequences(lines)
    X = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen=length, padding='post')
    return X

# Fungsi untuk prediksi terjemahan dengan model
def predict_seq(model, tokenizer, source):
    prediction = model.predict(source, verbose=0)[0]
    integers = [np.argmax(vector) for vector in prediction]
    target = [tokenizer.index_word.get(i, '') for i in integers if i != 0]
    return ' '.join(target)

# Fungsi untuk prediksi terjemahan (dari Bugis ke Indonesia)
def translate_text_with_model(translation_model, tokenizer, input_text):
    # Ensure input_text is treated as a list for encode_sequences
    input_seq = encode_sequences(translation_source_tokenizer, src_length, [input_text])
    # The predict_seq function expects a single sequence (batch size 1)
    translated_text = predict_seq(translation_model, translation_target_tokenizer, input_seq)
    return translated_text


# Pipeline untuk audio ke teks ke terjemahan
def audio_to_text_translation_pipeline(audio_file_path, stt_model, terjemahan_model, stt_tokenizer,
                                       translation_source_tokenizer, translation_target_tokenizer,
                                       src_length, tar_length):

    # Langkah 1: Prediksi teks Bugis dari file audio menggunakan model STT
    predicted_transcript = predict_audio_to_text(stt_model, audio_file_path)
    print("Transkripsi Bugis (STT Model):", predicted_transcript)

    # Langkah 2: Menerjemahkan teks Bugis ke dalam Bahasa Indonesia menggunakan model Terjemahan
    translated_text = translate_text_with_model(terjemahan_model,
                                                translation_target_tokenizer,
                                                predicted_transcript)
    print("Terjemahan Bahasa Indonesia (Translation Model):", translated_text)

    return predicted_transcript, translated_text

    """
    Pipeline to process audio file, transcribe to text, and translate the text.

    Args:
        audio_file_path (str): Path to the input audio file.
        stt_model: Loaded STT model.
        terjemahan_model: Loaded translation model.
        stt_tokenizer: Tokenizer for STT.
        translation_source_tokenizer: Tokenizer for the source language (Bugis).
        translation_target_tokenizer: Tokenizer for the target language (Indonesia).
        src_length (int): Source sequence length for translation model.
        tar_length (int): Target sequence length for translation model.

    Returns:
        tuple: A tuple containing the predicted Bugis transcript and the translated Indonesian text.
    """

# Fungsi untuk menghitung Word Error Rate (WER)
def calculate_wer(ground_truth, hypothesis):
    """
    Calculates the Word Error Rate (WER) between a ground truth and a hypothesis sentence.

    Args:
        ground_truth (str): The correct transcription.
        hypothesis (str): The predicted transcription.

    Returns:
        float: The WER score.
    """
    return wer(ground_truth, hypothesis)

# Fungsi untuk menghitung BLEU score
def calculate_bleu(ground_truth, hypothesis):
    """
    Calculates the BLEU score between a ground truth and a hypothesis sentence.

    Args:
        ground_truth (str): The correct translation (reference).
        hypothesis (str): The predicted translation.

    Returns:
        float: The BLEU score.
    """
    # sacrebleu expects ground_truth as a list of references
    return sacrebleu.sentence_bleu(hypothesis, [ground_truth]).score

# Fungsi untuk memuat data ground truth dari file CSV
def load_ground_truth_from_csv(csv_path):
    """
    Loads ground truth data from a CSV file.

    Args:
        csv_path (str): The path to the CSV file. The CSV should have columns
                        'filename', 'stt', and 'translation'.

    Returns:
        dict: A dictionary where keys are filenames and values are dictionaries
              containing 'stt' and 'translation' ground truths.
    """
    ground_truth_data = {}
    try:
        df = pd.read_csv(csv_path)
        for index, row in df.iterrows():
            filename = row['filename']
            stt_gt = row['stt']
            translation_gt = row['translation']
            ground_truth_data[filename] = {'stt': stt_gt, 'translation': translation_gt}
    except FileNotFoundError:
        print(f"Error: CSV file not found at {csv_path}")
    except KeyError as e:
        print(f"Error: Missing required column in CSV: {e}. Make sure the CSV has 'filename', 'stt', and 'translation' columns.")
    return ground_truth_data

# Fungsi untuk memproses file audio yang dipilih dan mengumpulkan hasil
def process_specific_audio_files_for_evaluation(list_of_audio_paths, ground_truth_data, stt_model, terjemahan_model, stt_tokenizer, translation_source_tokenizer, translation_target_tokenizer, src_length, tar_length):
    """
    Processes a list of specific audio files using the audio_to_text_translation_pipeline
    and collects predicted and ground truth texts for files with available ground truth.

    Args:
        list_of_audio_paths (list): A list of full paths to the audio files to process.
        ground_truth_data (dict): A dictionary where keys are filenames and values are dictionaries
                                  containing 'stt' and 'translation' ground truths.
        stt_model: Loaded STT model.
        terjemahan_model: Loaded translation model.
        stt_tokenizer: Tokenizer for STT.
        translation_source_tokenizer: Tokenizer for the source language (Bugis).
        translation_target_tokenizer: Tokenizer for the target language (Indonesia).
        src_length (int): Source sequence length for translation model.
        tar_length (int): Target sequence length for translation model.

    Returns:
        tuple: A tuple containing four lists (only for files with ground truth):
               - stt_ground_truths (list): List of STT ground truth texts.
               - stt_hypotheses (list): List of predicted STT texts.
               - translation_ground_truths (list): List of translation ground truth texts.
               - translation_hypotheses (list): List of predicted translation texts.
    """
    stt_ground_truths = []
    stt_hypotheses = []
    translation_ground_truths = []
    translation_hypotheses = []

    print("\n--- Processing Selected Audio Files ---")
    for file_path in list_of_audio_paths:
        filename = os.path.basename(file_path) # Extract filename from path
        print(f"\nProcessing file: {filename}")

        # Check if ground truth exists for this file before processing
        if filename in ground_truth_data:
            try:
                # Use the pipeline function
                predicted_transcript, translated_text = audio_to_text_translation_pipeline(
                    file_path,
                    stt_model,
                    terjemahan_model,
                    stt_tokenizer,
                    translation_source_tokenizer,
                    translation_target_tokenizer,
                    src_length,
                    tar_length
                )
                print(f"Processed successfully: {filename}")

                # Collect ground truth and hypothesis
                stt_ground_truths.append(ground_truth_data[filename]['stt'])
                stt_hypotheses.append(predicted_transcript)
                translation_ground_truths.append(ground_truth_data[filename]['translation'])
                translation_hypotheses.append(translated_text)

            except Exception as e:
                print(f"Error processing {filename}: {e}")
        else:
            print(f"Warning: No ground truth data found for {filename}. Skipping processing and evaluation for this file.")


    return stt_ground_truths, stt_hypotheses, translation_ground_truths, translation_hypotheses

## 4. Contoh Penggunaan Pipeline

Contoh penggunaan pipeline untuk memproses satu file audio.

In [ ]:
# Contoh penggunaan pipeline
# Gantilah dengan path file audio yang sesuai di Google Drive Anda
audio_file_path_example = '/content/drive/MyDrive/databaru/dataAudio16Hz/50_copy_1.wav'

print(f"Processing audio file: {audio_file_path_example}")

# Pastikan model dan tokenizer sudah dimuat
if 'stt_model' in globals() and 'terjemahan_model' in globals() and 'stt_tokenizer' in globals(
    ) and 'translation_source_tokenizer' in globals() and 'translation_target_tokenizer' in globals():
    predicted_transcript_example, translated_text_example = audio_to_text_translation_pipeline(
        audio_file_path_example,
        stt_model,
        terjemahan_model,
        stt_tokenizer,
        translation_source_tokenizer,
        translation_target_tokenizer,
        src_length,
        tar_length
    )

    print(f"\n--- Hasil Pipeline ---")
    print(f"Transkripsi Bugis: {predicted_transcript_example}")
    print(f"Terjemahan Bahasa Indonesia: {translated_text_example}")
else:
    print("Model atau tokenizer belum dimuat. Harap jalankan kembali bagian 2.")

Processing audio file: /content/drive/MyDrive/databaru/dataAudio16Hz/50_copy_1.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
Transkripsi Bugis (STT Model): mitauka jokka dare'e cilaleku
Terjemahan Bahasa Indonesia (Translation Model): saya pergi pergi kebun diri mahal

--- Hasil Pipeline ---
Transkripsi Bugis: mitauka jokka dare'e cilaleku
Terjemahan Bahasa Indonesia: saya pergi pergi kebun diri mahal


## 5. Evaluasi Performansi

Memuat data ground truth, memproses file audio terpilih, dan menghitung metrik WER dan BLEU.

In [ ]:
# Step 5: Memuat ground truth dari multiple CSV
# Define the paths to your two ground truth CSV files
ground_truth_csv_paths = [
    '/content/drive/MyDrive/skripsi/Book2groundtht.csv',
    '/content/drive/MyDrive/skripsi/transkripbarutanpawav.csv',
    '/content/drive/MyDrive/databaru/groundtruthpengujian.csv',
    '/content/drive/MyDrive/databaru/groundtruthpengujian.csv'
]

# Load the ground truth data from the list of CSV files
combined_ground_truth_data = {}
for csv_path in ground_truth_csv_paths:
    ground_truth_data_single_csv = load_ground_truth_from_csv(csv_path)
    combined_ground_truth_data.update(ground_truth_data_single_csv)

print(f"Loaded and combined ground truth data from {len(ground_truth_csv_paths)} CSV files.")
print(f"Total entries in combined ground truth data: {len(combined_ground_truth_data)}")


# Step 6: Membuat daftar path audio yang akan dievaluasi
# Define a list of specific audio file paths to evaluate
selected_audio_files = [
    '/content/drive/MyDrive/databaru/dataUji/datauji11.wav',
    '/content/drive/MyDrive/databaru/dataUji/datauji12.wav',
    '/content/drive/MyDrive/databaru/dataUji/datauji13.wav',
    '/content/drive/MyDrive/databaru/dataUji/datauji14.wav',
    '/content/drive/MyDrive/databaru/dataUji/datauji15.wav',
    '/content/drive/MyDrive/databaru/dataUji/datauji21.wav',
    '/content/drive/MyDrive/databaru/dataUji/datauji22.wav',
    '/content/drive/MyDrive/databaru/dataUji/datauji23.wav',
    '/content/drive/MyDrive/databaru/dataUji/datauji24.wav',
    '/content/drive/MyDrive/databaru/dataUji/datauji25.wav',
    '/content/drive/MyDrive/databaru/dataUji/datauji31.wav',
    '/content/drive/MyDrive/databaru/dataUji/datauji32.wav',
    '/content/drive/MyDrive/databaru/dataUji/datauji33.wav',
    '/content/drive/MyDrive/databaru/dataUji/datauji34.wav',
    '/content/drive/MyDrive/databaru/dataUji/datauji35.wav',
    '/content/drive/MyDrive/databaru/dataUji/uji1.wav',
    '/content/drive/MyDrive/databaru/dataUji/uji2.wav',
    '/content/drive/MyDrive/databaru/dataUji/uji3.wav',
    '/content/drive/MyDrive/databaru/dataUji/uji4.wav',
    '/content/drive/MyDrive/databaru/dataUji/uji5.wav'
]

print(f"\nDefined a list with {len(selected_audio_files)} audio file paths to evaluate.")

# Process the selected audio files and collect results
# Ensure model and tokenizer are loaded
if 'stt_model' in globals() and 'terjemahan_model' in globals() and 'stt_tokenizer' in globals() and 'translation_source_tokenizer' in globals() and 'translation_target_tokenizer' in globals():
    stt_gts, stt_hyps, trans_gts, trans_hyps = process_specific_audio_files_for_evaluation(
        selected_audio_files,
        combined_ground_truth_data,
        stt_model,
        terjemahan_model,
        stt_tokenizer,
        translation_source_tokenizer,
        translation_target_tokenizer,
        src_length,
        tar_length
    )

    print(f"\nCollected {len(stt_gts)} STT entries with ground truth.")
    print(f"Collected {len(trans_gts)} Translation entries with ground truth.")


    # Step 7: Menghitung dan menampilkan WER dan BLEU
    print("\n--- Evaluation Results ---")

    # Hitung dan tampilkan WER rata-rata
    if stt_gts:
        average_wer = sum([calculate_wer(gt, hp) for gt, hp in zip(stt_gts, stt_hyps)]) / len(stt_gts)
        print(f"Average WER for STT: {average_wer:.4f}")
    else:
        print("No STT data available for WER calculation (no selected files had ground truth).")

    # Hitung dan tampilkan BLEU rata-rata
    if trans_gts:
        # sacrebleu.corpus_bleu expects references as a list of lists
        average_bleu = sacrebleu.corpus_bleu(trans_hyps, [trans_gts]).score
        print(f"Corpus BLEU for Translation: {average_bleu:.2f}")
    else:
        print("No Translation data available for BLEU calculation (no selected files had ground truth).")
else:
    print("Model atau tokenizer belum dimuat. Harap jalankan kembali bagian 2.")

Loaded and combined ground truth data from 4 CSV files.
Total entries in combined ground truth data: 3520

Defined a list with 20 audio file paths to evaluate.

--- Processing Selected Audio Files ---

Processing file: datauji11.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
Transkripsi Bugis (STT Model): tennang de'to nasaba ni ni kareba
Terjemahan Bahasa Indonesia (Translation Model): perasaanku karena karena karena
Processed successfully: datauji11.wav

Processing file: datauji12.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
Transkripsi Bugis (STT Model): de' no pasa'e melli
Terjemahan Bahasa Indonesia (Translation Model): saya saya beli
Processed successfully: datauji12.wav

Processing file: datauji13.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
Transkripsi Bugis (STT Model): pake mitai mebbu beppa
Terjemahan Bahasa Indonesia (Translation Model): saya ingin membuat membuat
Processed successfully: datauji13.wav

Processing file: datauji14.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
Transkripsi Bu

## 6. Finish task